<a href="https://colab.research.google.com/github/jmisf14/Predicting-UFC-Fight-Outcomes/blob/main/UFCPredictor_Jos%C3%A9MiguelSerrano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting-UFC-Fight-Outcomes


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")

In [3]:
# Load datasets
results = pd.read_csv("/content/ufc_fight_results.csv")
stats = pd.read_csv("/content/ufc_fight_stats.csv")
fighters = pd.read_csv("/content/ufc_fighters.csv")


In [4]:
results.head()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,L/W,Welterweight Bout,Submission,4,1:39,5 Rnd (5-5-5-5-5),Jason Herzog,Guillotine Choke From Mount,http://ufcstats.com/fight-details/371ed8497799...
1,UFC Fight Night: Edwards vs. Brady,Jan Blachowicz vs. Carlos Ulberg,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Marc Goddard,Ben Cartlidge 28 - 29.David Lethaby 28 - 29.Vi...,http://ufcstats.com/fight-details/6d9ea9f436ed...
2,UFC Fight Night: Edwards vs. Brady,Gunnar Nelson vs. Kevin Holland,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,Darryl Ransom 28 - 29.David Lethaby 28 - 29.An...,http://ufcstats.com/fight-details/96c6be6b2238...
3,UFC Fight Night: Edwards vs. Brady,Molly McCann vs. Alexia Thainara,L/W,Women's Strawweight Bout,Submission,1,4:32,3 Rnd (5-5-5),Lukasz Bosacki,Rear Naked Choke,http://ufcstats.com/fight-details/b5b4b69bccb9...
4,UFC Fight Night: Edwards vs. Brady,Jordan Vucenic vs. Chris Duncan,L/W,Lightweight Bout,Submission,2,3:42,3 Rnd (5-5-5),Jason Herzog,Guillotine Choke After Drop to Guard,http://ufcstats.com/fight-details/8e6640c007ad...


In [5]:
stats.head()

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,Round 1,Leon Edwards,0.0,20 of 29,68%,20 of 29,0 of 0,---,0.0,0.0,0:15,8 of 16,11 of 12,1 of 1,8 of 17,12 of 12,0 of 0
1,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,Round 2,Leon Edwards,0.0,0 of 0,---,0 of 0,0 of 0,---,0.0,0.0,0:00,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0
2,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,Round 3,Leon Edwards,0.0,3 of 4,75%,3 of 4,0 of 1,0%,0.0,0.0,0:00,2 of 3,0 of 0,1 of 1,3 of 4,0 of 0,0 of 0
3,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,Round 4,Leon Edwards,0.0,0 of 2,0%,0 of 2,0 of 0,---,0.0,0.0,0:00,0 of 2,0 of 0,0 of 0,0 of 2,0 of 0,0 of 0
4,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,Round 1,Sean Brady,0.0,19 of 39,48%,47 of 69,1 of 1,100%,0.0,0.0,0:54,10 of 27,6 of 6,3 of 6,13 of 33,6 of 6,0 of 0


In [6]:
fighters.head()

,First Name,Last Name,Nickname,Height,Weight,Reach,Stance,Wins,Losses,Draws
0,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5,3,0
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0
2,Nariman,Abbasov,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0
3,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0
4,Hamdy,Abdelwahab,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0


## Data Cleaning

### Clean fighters

In [7]:
# Combine first and last name
fighters["First Name"] = fighters["First Name"].fillna("").str.strip()
fighters["Last Name"] = fighters["Last Name"].fillna("").str.strip()
fighters["fighter_name"] = (fighters["First Name"] + " " + fighters["Last Name"]).str.strip()

# Optional: clean nickname
fighters["Nickname"] = fighters["Nickname"].fillna("Unknown")

# Drop unnecessary name parts
fighters.drop(columns=["First Name", "Last Name"], inplace=True)

fighters.head()

,Nickname,Height,Weight,Reach,Stance,Wins,Losses,Draws,fighter_name
0,Unknown,--,155 lbs.,--,NaN,5,3,0,Tom Aaron
1,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,Danny Abbadi
2,Bayraktar,"5' 8""",155 lbs.,"66.0""",Orthodox,28,4,0,Nariman Abbasov
3,Tank,"6' 0""",265 lbs.,--,Switch,10,15,0,David Abbott
4,The Hammer,"6' 2""",264 lbs.,"72.0""",Southpaw,5,0,0,Hamdy Abdelwahab


### Clean results

In [8]:
# Split fighter names from 'bout'
results[["fighter_A", "fighter_B"]] = results["BOUT"].str.split(" vs. ", expand=True)

# Melt into long format: 1 row per fighter
long_results = pd.melt(
    results,
    id_vars=[col for col in results.columns if col not in ["fighter_A", "fighter_B"]],
    value_vars=["fighter_A", "fighter_B"],
    var_name="corner", value_name="fighter_name"
)
results.drop(columns=["URL"], inplace=True)
long_results["corner"] = long_results["corner"].map({"fighter_A": "Red", "fighter_B": "Blue"})
results.head()

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,fighter_A,fighter_B
0,UFC Fight Night: Edwards vs. Brady,Leon Edwards vs. Sean Brady,L/W,Welterweight Bout,Submission,4,1:39,5 Rnd (5-5-5-5-5),Jason Herzog,Guillotine Choke From Mount,Leon Edwards,Sean Brady
1,UFC Fight Night: Edwards vs. Brady,Jan Blachowicz vs. Carlos Ulberg,L/W,Light Heavyweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Marc Goddard,Ben Cartlidge 28 - 29.David Lethaby 28 - 29.Vi...,Jan Blachowicz,Carlos Ulberg
2,UFC Fight Night: Edwards vs. Brady,Gunnar Nelson vs. Kevin Holland,L/W,Welterweight Bout,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Daniel Movahedi,Darryl Ransom 28 - 29.David Lethaby 28 - 29.An...,Gunnar Nelson,Kevin Holland
3,UFC Fight Night: Edwards vs. Brady,Molly McCann vs. Alexia Thainara,L/W,Women's Strawweight Bout,Submission,1,4:32,3 Rnd (5-5-5),Lukasz Bosacki,Rear Naked Choke,Molly McCann,Alexia Thainara
4,UFC Fight Night: Edwards vs. Brady,Jordan Vucenic vs. Chris Duncan,L/W,Lightweight Bout,Submission,2,3:42,3 Rnd (5-5-5),Jason Herzog,Guillotine Choke After Drop to Guard,Jordan Vucenic,Chris Duncan
